In [84]:
# -*- coding: utf-8 -*-
# From the public delegates dossier here: https://static1.squarespace.com/static/5640b90ce4b0bd41b3442125/t/5aafda2870a6ad1c9302c45d/1521474150768/DELEGATE.DOSSIER.pdf
import pandas as pd, numpy as np, ast
import matplotlib.pyplot as plt
from textblob import TextBlob 

In [85]:
#Generating names and bios table from raw text file
names = []
affiliation = []
bios = []
curtext = ""
file = open("ygs_delegates_raw.txt","r")
count = 0

while True:
    line = file.readline()
    if not line:
        break
    if line == '\n':
        if curtext != '':
            bios = bios + [curtext.strip()]
        curtext = ""
        count = 0
        continue
    if count == 0:
        names = names + [line.strip()]
        count = count + 1
    elif count == 1:
        affiliation = affiliation + [line.strip()]
        count = count + 1
    elif count == 2:
        curtext = curtext + line
        
dat = pd.DataFrame({"Names": names, "Affiliations": affiliation, "Bios": bios})
dat = dat[["Names", "Affiliations", "Bios"]]
kw = []
dat['keywords'] = pd.Series("", index=dat.index)
for i in range(dat.shape[0]):
    nf = TextBlob(dat.iloc[i]["Bios"].decode('utf-8')).noun_phrases
    dat.loc[i,"keywords"] = nf
dat.to_csv('ygsdelegates.csv') #have do to additional processing on the actual document to make it utf8-compatible

In [86]:
dat = pd.DataFrame.from_csv('ygsdelegates.csv')
dat

,Names,Affiliations,Bios,keywords
0,Haoyin Wu,Student at IHEID,"Born in Guangzhou, studied in Beijing and live...","[u'born', u'guangzhou', u'beijing', u'singapor..."
1,Iris Blom,"Schwartzman Scholar, Tsinghua University",Iris Blom graduated in July 2017 from The Univ...,"[u'iris blom', u'july', u'amsterdam', u'bachel..."
2,Viola Rothschild,"China Studies Research Associate, Council on F...",Viola Rothschild is a China studies research a...,"[u'viola rothschild', u'china', u'studies rese..."
3,Md Nuruzzaman Khan,"Filmmaker, Rosa Production Slovenia",Nuruzzaman Khan is a documentary filmmaker. Kh...,"[u'nuruzzaman khan', u'documentary filmmaker',..."
4,Vineet Mittal,"Master's Candidate, London School of Economics...",Vineet Mittal is pursuing a double degree at t...,"[u'vineet mittal', u'double degree', u'london'..."
5,Carmina Mancenon,Yale MBA/MA International Relations Candidate,Carmina Mancenon is a Filipino raised in Tokyo...,"[u'carmina mancenon', u'filipino', u'tokyo', u..."
6,Hang Fu,Capstone Consultant of Google,Hang Fu is currently the capstone consultant f...,"[u'hang fu', u'capstone consultant', u'google ..."
7,Pamir Hassas,Youth Activist and Parliament Employee,Pamir Hassas is from Afghanistan born in April...,"[u'pamir hassas', u'afghanistan', u'april', u""..."
8,YUE ZHANG,Postgraduate of Shandong University,Yue is currently a postgraduate major in Inter...,"[u'yue', u'postgraduate major', u'politics', u..."
9,Luoyi Zhou,Barcelona Institute of International Studies,Luoyi (Natalia) is currently pursuing her Mast...,"[u'luoyi', u'natalia', u'master\u2019s', u'chi..."


In [64]:
#Preparing the keywords list. There is a lot of manuel processing for this section so its meant to run in parallel, not procedurally after the above.

# kwlist = []
# for i in range(dat.shape[0]):
#     nf = TextBlob(dat.iloc[i]["Bios"].decode('utf-8')).noun_phrases
#     kwlist = kwlist + nf
#     kwlist = list(set(kwlist))
# print kwlist

kw = {}
for i in range(dat.shape[0]):
    name = dat.iloc[i]["Names"].decode('utf-8')
    nf = TextBlob(dat.iloc[i]["Bios"].decode('utf-8')).noun_phrases
    kw[name] = nf

keywords = {}
for i in range(dat.shape[0]):
    name = dat.iloc[i]["Names"].decode('utf-8')
    for word in kw[name]:
        if (not keywords.has_key(word)):
            keywords[word] = set([name])
        else:
            keywords[word].add(name) 

keywords_red = {}
for word in keywords:
    if (len(keywords[word])>1):
        keywords_red[word] = keywords[word]
        
#list(keywords_red.keys())

In [90]:
#Preparing the table with reduced keywords for manual editing
#kw_red is from manuel processing of all the keywords from above, read in from file 'ygs_keywords_reduced1.0.txt'
kw_red = []
with open('ygs_keywords_reduced1.0.txt') as fp:
    line = fp.readline()
    kw_red = ast.literal_eval(line)

In [98]:
dat_kwred = dat.copy()
for i in range(dat_kwred.shape[0]):
    less_kw = [n for n in ast.literal_eval(dat.loc[i,"keywords"]) if n in kw_red]
    dat_kwred.loc[i,"keywords"] = less_kw
dat_kwred.to_csv('ygsdelegates_kw_reduced.csv') 

In [162]:
dat_final = pd.DataFrame.from_csv('ygsdelegates_kw_reduced_edit.csv')

keywords = {}
for i in range(dat_final.shape[0]):
    name = dat.iloc[i]["Names"].decode('utf-8')
    cur_kw = ast.literal_eval(dat_final.loc[i,"keywords"])
    for word in cur_kw:
        if (not keywords.has_key(word)):
            keywords[word] = set([name]) #add names as a set becuase some names might have the same keyword repeated in its list
        else:
            keywords[word].add(name) 

keywords_red = {}
keywords_solo = []
for word in keywords:
    if (len(keywords[word])>1):
        keywords_red[word] = keywords[word]
    else:
        keywords_solo += [word]

keywords_red = {key : list(keywords_red[key]) for key in keywords_red.keys()} #change from set to list for indexing later on

In [166]:
table = []

for key in keywords_red.keys():
    for i in range(len(keywords_red[key])):
        for j in range(len(keywords_red[key])):
            if i != j:
                table = table + [[keywords_red[key][i], keywords_red[key][j], key]]
network = pd.DataFrame(data=table, columns = ['Delegate 1', 'Delegate 2', 'Connection'])
network.to_csv('final_ygsdelegates_connections.csv',encoding='utf8', index = False)

In [164]:
network

,Delegate 1,Delegate 2,Connection
0,Bhargav Sriganesh,Ahtziri Gonzalez,lse
1,Bhargav Sriganesh,Yangyang Chen,lse
2,Ahtziri Gonzalez,Bhargav Sriganesh,lse
3,Ahtziri Gonzalez,Yangyang Chen,lse
4,Yangyang Chen,Bhargav Sriganesh,lse
5,Yangyang Chen,Ahtziri Gonzalez,lse
6,Patrick Farrell,Marie-Therese Png,social science
7,Patrick Farrell,Alex Jiya,social science
8,Patrick Farrell,Kazuki Miyazaki,social science
9,Patrick Farrell,Matthew Pierri,social science
